In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import getpass

In [5]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

 ········


In [6]:
model = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp",
)

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?'

In [9]:
tools = []
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model,tools)

In [12]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"][1]

AIMessage(content='Hi there! How can I help you today? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-797f0aad-9131-4e9e-b762-12b97e1e74c3-0', usage_metadata={'input_tokens': 3, 'output_tokens': 12, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

In [29]:
dataset_context = '''

Price and Price Relationships
High_n, Low_n, Open_n, Close_n: High, low, opening, and closing prices for period n.
High_n-Low_n: Range between high and low prices in period n.
Open_n-Close_n: Difference between opening and closing prices in period n.

Simple Moving Averages (SMAs)
SMA_10, SMA_20: 10-period and 20-period Simple Moving Averages.
SMA_20-SMA_10: Difference between 20-period and 10-period SMAs.

Change Length Indicators
Variables representing the periods over which changes occur:
Open_n_changelen, High_n_changelen, Low_n_changelen, Close_n_changelen: Change length for opening, high, low, and closing prices.
SMA_20-SMA_10_changelen: Change length of SMA differences.
Close_n_slope_3_changelen, Close_n_slope_5_changelen, Close_n_slope_10_changelen: Change length of closing price slopes over 3, 5, and 10 periods.

Price Slopes
Close_n_slope_3, Close_n_slope_5, Close_n_slope_10: Slopes (rates of change) of the closing price over 3, 5, and 10 periods.

Valuation Variables
Variables representing absolute values of indicators or prices:
Open_n_val, High_n_val, Low_n_val, Close_n_val: Absolute values for opening, high, low, and closing prices.
SMA_10_val, SMA_20_val: Absolute values of SMAs.
CMO_14_val, RSI_14_val: Absolute values of momentum and strength indicators.

Technical Indicators
VOL_SMA_20: 20-period Simple Moving Average of volume.
RSI_14: Relative Strength Index (momentum) over 14 periods.
CMO_14: Chande Momentum Oscillator over 14 periods.
BBL_5_2.0, BBM_5_2.0, BBU_5_2.0: Bollinger Bands (Lower, Middle, and Upper) over 5 periods with 2.0 standard deviation.
BBB_5_2.0, BBP_5_2.0: Bollinger Band Width and Percentage.
MACD_12_26_9: Moving Average Convergence Divergence line.
MACDh_12_26_9, MACDs_12_26_9: MACD histogram and signal line.

VWAP and Momentum
VWAP_D: Volume Weighted Average Price for the day.
MOM_30: Momentum indicator over 30 periods.

Targets
target_5, target_10: Targets for 5-period and 10-period predictions.
target_5_val, target_10_val: Numerical values of these targets.

Miscellaneous
row_num: Row index or identifier.
era: Period grouping (e.g., week, month, year).
id: Unique identifier for data rows.

Following code structure should be followed:
def exponential_moving_average_prediction(df, span=15):
    """
    Calculate predictions for target_5 and target_10 based on EMA and other indicators.
    Uses pre-calculated values from the dataframe.
    
    Args:
        df: DataFrame containing the price data and indicator values
        span: EMA span parameter
    
    Returns:
        tuple: (target_5_prediction, target_10_prediction) between -1 and 1
    """
    # Get latest row of data
    current = df.iloc[-1]
    
    # Initialize prediction scores
    score_5 = 0
    score_10 = 0
    
    ### Code here

    ### End of code

    return score_5, score_10
'''

SYSTEM_PROMPT = f"""
You are best at writing python code and have good knowledge about finance. 
You'll assist people in converting their trading strategies into code.
You are provided with dataset containing information about financial and technical indicators.

Available dataset context:
{dataset_context}

Strictly return python code only when Human gives a strategy or asks to change the previous strategy.
If the instruction is ambiguous ask for more detailed strategy else reply generally.

"""

In [48]:
from langchain_core.tools import tool
i
@tool
def write_strategy(strategy:string) -> string:
    '''
        Write Python code for a trading strategy
    '''

    return f"Here's the Python code for your strategy:\n\n{strategy}"

tools = [write_strategy]

NameError: name 'string' is not defined

In [44]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model,tools,checkpointer=memory,state_modifier=SYSTEM_PROMPT)
config = {"configurable":{"thread_id":"abc123"}}

In [45]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]},config)
response['messages'][-1].content

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.tools[0].function_declarations[0].parameters.properties[strategy].type: must be specified when not using one_of


In [42]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Write me a trading strategy that involves the use of slopes!")]},config)
response['messages'][-1].content

'Okay, let\'s craft a trading strategy using price slopes. Here\'s a concept that combines multiple slope indicators and other factors for potentially stronger signals:\n\n**Strategy Name:** Slope Confirmation Strategy\n\n**Core Idea:** This strategy aims to identify potential price reversals or continuations by analyzing the short-term and medium-term slopes of the closing price. It also incorporates the SMA to identify the trend.\n\n**Indicators:**\n\n1.  **Short-Term Slope:** `Close_n_slope_3` (Slope of closing price over 3 periods)\n2.  **Medium-Term Slope:** `Close_n_slope_10` (Slope of closing price over 10 periods)\n3.  **Simple Moving Average (SMA):** `SMA_20` (20-period SMA)\n4. **SMA Difference:** `SMA_20-SMA_10` (Difference between 20 and 10 period SMA)\n\n**Entry Conditions:**\n\n*   **Long Entry:**\n    *   `Close_n_slope_3` is positive, indicating a short-term uptrend.\n    *   `Close_n_slope_10` is positive, confirming a medium-term uptrend.\n    *   `SMA_20-SMA_10` is p